In [1]:
# Разделяем диаметр и толщину стенки и чистим их
def split_d(x):
    x = str(x)
    x = x.replace(',', '.')
    return re.split('х|Х|x|X', x)[0]

def split_s(x):
    x = str(x)
    x = x.replace(',', '.')
    try:
        return re.split('х|Х|x|X', x)[1]
    except:
        return x
    
def clean_d_s(x):
    x = str(x)
    x = x.replace('п', '')
    x = x.replace('П', '')
    x = x.replace('nan', '')
    x = x.replace(',', '.')
    return x

In [2]:
def clean_steps(x):
    x = str(x)
    x = x.replace(u'X', '/')
    x = x.replace(u'x', '/')
    x = x.replace(u'Х', '/')
    x = x.replace(u'х', '/')
    x = x.replace(',', '.')
    x = x.replace('\\', '/')
    x = x.replace('/-', '/0')
    x = x.replace('-/', '0/')
    x = x.replace('***', '0')
    x = x.replace('*', '0')
    x = x.replace('-', '/')
    x = x.replace('nan', '')
    return x

def convert_step0(x):
    return clean_steps(x).split('/')[0]
   
def convert_step1(x):
    try:
        return clean_steps(x).split('/')[1]
    except:
        return x

In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
df = pd.read_excel('20-30.xlsx')

In [5]:
df.columns

Index(['Номер пакета', 'Дата', 'цикл', 'тз', 'то', 'Номер партии',
       'Номер\nплавки', 'Обозна-чение\nНД', 'марка стали', 'кол-во, шт.',
       'масса\nтн.', 'Размер труб, мм', 'время начала т/о', 'шаг балок, сек',
       '1 зона закалка', '2 зона закалка', '3 зона закалка',
       'скорость спрейера, м/с', 't˚ C трубы\nпосле спреера',
       'закалочная ванна', 't˚ C трубы\nпосле ванны', '1 зона отпуск',
       '2 зона отпуск', '3 зона отпуск', '4 зона отпуск', '5 зона отпуск',
       '№ пакета', '№ партии', 'кол-во, шт', 'масса\nтн',
       'Обозначение НД, гр. проч-ности'],
      dtype='object')

In [6]:
df = df.groupby(['№ партии','Номер\nплавки', 'марка стали', 'Размер труб, мм','шаг балок, сек'])['масса\nтн'].sum()

In [7]:
df = df.reset_index()

In [8]:
df[u'шаг балок закалочная печь, сек'] = df['шаг балок, сек'].apply(convert_step0)
df[u'шаг балок отпускная печь, сек'] = df['шаг балок, сек'].apply(convert_step1)
df['диаметр'] = df['Размер труб, мм'].apply(split_d)
df['толщина стенки'] = df['Размер труб, мм'].apply(split_s)
df['толщина стенки'] = df['толщина стенки'].apply(clean_d_s)
df['толщина стенки'] = df['толщина стенки'].apply(lambda x: x.replace('107.95', ''))
df['диаметр'] = df['диаметр'].apply(clean_d_s)

In [9]:
del df['шаг балок, сек']
del df['Размер труб, мм']

In [10]:
df.to_excel('june_econom_10.xlsx')

In [11]:
df = df.groupby(['марка стали','толщина стенки','диаметр', 'шаг балок закалочная печь, сек', 'шаг балок отпускная печь, сек'])['масса\nтн'].sum()
df = df.reset_index()
df.to_excel('june_groups_10.xlsx')